In [ ]:
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
import random
import logging
import time
import pdb

In [ ]:
# Creating log file
def create_logger(output_path, cfg_name):
    log_file = '{}_{}.log'.format(cfg_name, time.strftime('%Y-%m-%d-%H-%M'))
    head = '%(asctime)-15s %(message)s'
    logging.basicConfig(filename=os.path.join(output_path, log_file), format=head)
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    
    return logger

# Loading and precessing data
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path, '%s-labels.idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images.idx3-ubyte' % kind)
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII', imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)

    return images, labels

In [ ]:
# Parts of model
def initialize_parameters(layers):
    # Initialize parameters according to different types of layers
    new_layers = []
    for i, layer in enumerate(layers):
        mode = layer['mode'] # 'fc'
        if mode == 'fc':
            n_now = layer['n_now']
            n_prev = layer['n_prev']
            layer['W']=(np.random.rand(n_now, n_prev) - 0.5) * 0.2 # random sample in [-0.1, 0.1] 
            layer['b']=np.zeros((n_now,1))
            layer['dW']=np.zeros_like(layer['W'])
            layer['db']=np.zeros_like(layer['b'])
        else:
            print('Wrong layer in [{}]'.format(i))
        new_layers.append(layer)
            
    return new_layers

def sigmoid(Z):
    # Sigmoid activation function
    A = 1/(1+np.exp(-Z))
    cache = Z
    return A, cache

def sigmoid_backward(dA, cache):
    # Backpropogation of sigmoid activation function
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    return dZ

def relu(Z):
    # Relu activation function
    A = np.maximum(0,Z)
    cache = Z
    return A, cache

def relu_backward(dA, cache):
    # Backpropogation of Relu activation function 
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    dZ[Z < 0] = 0
    return dZ

def softmax(Z):
    # Softmax activation function
    n, m = Z.shape
    A = np.exp(Z)
    A_sum = np.sum(A, axis = 0)
    A_sum = A_sum.reshape(-1, m)
    A = A / A_sum
    cache = Z
    return A, cache

def softmax_backward(A, Y):
    # Backpropogation of softmax activation function
    # loss = - ln a[j] (y[j] = 1, j = {0, ..., n}) 
    m = A.shape[1]
    dZ = (A - Y) / np.float(m)
    return dZ

def linear_activation_forward(A_prev, layer, activation='relu'):
    W = layer['W']
    b = layer['b']
    if activation=='sigmoid':
        Z, linear_cache=np.dot(W, A_prev)+b, (A_prev, W, b)
        A, activation_cache=sigmoid(Z)
    elif activation=='relu':
        Z, linear_cache=np.dot(W, A_prev)+b, (A_prev, W, b)
        A, activation_cache=relu(Z)
    else:
        Z = np.dot(W, A_prev)+b
        A = Z
    return A, Z

def linear_activation_backward(dA, layer, activation):
    # Backward propagatIon module - linear activation backward
    A_prev = layer['A_prev']
    W = layer['W']
    b = layer['b']
    Z = layer['Z']
    if activation=='relu':
        dZ=relu_backward(dA, Z)
    elif activation=='sigmoid':
        dZ=sigmoid_backward(dA, Z)
    else:
        dZ = dA 
    n, m = dA.shape
    dA_prev=np.dot(W.T, dZ)
    dW = np.dot(dZ, A_prev.T)
    db = np.sum(dZ, axis = 1).reshape(n,1)
    
    return dA_prev, dW, db

In [ ]:
# Constructing THREE layers BP neural networks
def forward_propogation(X, layers):
    m = X.shape[1]
    # -1- fully connected layer
    layers[0]['A_prev'] = X
    A, Z = linear_activation_forward(X, layers[0], activation='relu')
    layers[0]['Z'] = Z
    
    # -2- fully connected layer
    layers[1]['A_prev'] = A
    A, Z = linear_activation_forward(A, layers[1], activation='relu')
    layers[1]['Z'] = Z

    # -3- fully connected layer
    layers[2]['A_prev'] = A
    _, Z = linear_activation_forward(A, layers[2], activation='none')
    layers[2]['Z'] = Z
    AL, _ = softmax(Z)

    return AL, layers

def backward_propogation(AL, Y, layers):
    m = Y.shape[1]
    # -3- fully connected layer
    dZ = softmax_backward(AL, Y)
    dA_prev, dW, db = linear_activation_backward(dZ, layers[2], 'none')
    layers[2]['dW'] = dW
    layers[2]['db'] = db
    
    # -2- fully connected layer
    dA_prev, dW, db = linear_activation_backward(dA_prev, layers[1], 'relu')
    layers[1]['dW'] = dW
    layers[1]['db'] = db

    # -1- fully connected layer
    dA_prev, dW, db = linear_activation_backward(dA_prev, layers[0], 'relu')
    layers[0]['dW'] = dW
    layers[0]['db'] = db

    return layers

def update_parameters(layers, learning_rate):
    num_layer = len(layers)
    for i in range(num_layer):
        mode = layers[i]['mode'] # 'fc'
        if mode == 'fc':
            layers[i]['W'] = layers[i]['W'] - learning_rate*layers[i]['dW']
            layers[i]['b'] = layers[i]['b'] - learning_rate*layers[i]['db']
        else:
            print('Wrong layer mode in [{}]'.format(i))

    return layers

def predict(X_test, Y_test, layers):
    m = X_test.shape[1]
    n = Y_test.shape[0]
    pred = np.zeros((n,m))
    pred_count = np.zeros((n,m)) - 1 # for counting accurate predictions 
    
    # forward propagation
    AL, _ = forward_propogation(X_test, layers)

    # convert prediction to 0/1 form
    max_index = np.argmax(AL, axis = 0)
    pred[max_index, list(range(m))] = 1
    pred_count[max_index, list(range(m))] = 1
    
    accuracy = np.float(np.sum(pred_count == Y_test)) / m
    
    return pred, accuracy

def compute_accuracy(AL, Y):
    n, m = Y.shape
    pred_count = np.zeros((n,m)) - 1
    
    max_index = np.argmax(AL, axis = 0)
    pred_count[max_index, list(range(m))] = 1
    
    accuracy = np.float(np.sum(pred_count == Y)) / m
    
    return accuracy

def compute_cost(AL, Y):
    n, m = Y.shape
    cost = - np.sum(np.log(AL) * Y) / m
    cost=np.squeeze(cost)

    return cost

def train_mini_batch(X_train, Y_train, X_test, Y_test, layers, logger, batch_size=10, num_epoch=1, learning_rate=0.01):
    logger.info('------------ Train BPNN with mini batch ------------')
    logger.info('Initial weights: FC [-0.1, 0.1], Mapping [-0.1, 0.1]')
    logger.info('Initial bias: FC 0')
    logger.info('Batch size: {}'.format(batch_size))
    logger.info('Learning rate: {}'.format(learning_rate))
    
    # number of iteration
    num_sample=X_train.shape[0]
    num_iteration = num_sample // batch_size
    index = list(range(num_sample))
    
    for epoch in range(num_epoch):
        losses = []
        accuracies = []
        # random.shuffle(index) # random sampling every epoch
        for iteration in range(num_iteration):
            batch_start = iteration * batch_size
            batch_end = (iteration + 1) * batch_size
            if batch_end > num_sample:
                batch_end = num_sample
            X_train_batch = (X_train[index[batch_start:batch_end]]).T
            Y_train_batch = (Y_train[index[batch_start:batch_end]]).T
            AL, layers = forward_propogation(X_train_batch, layers)
            loss = compute_cost(AL, Y_train_batch)
            accuracy = compute_accuracy(AL, Y_train_batch)
            layers = backward_propogation(AL, Y_train_batch, layers)
            layers = update_parameters(layers, learning_rate)
            losses.append(loss)
            accuracies.append(accuracy)
            if (iteration+1) % 600 == 0:
                logger.info('Epoch [{}] Iteration [{}]: loss = {} accuracy = {}'.format(epoch, iteration+1, loss, accuracy))
                print('Epoch [{}] Iteration [{}]: loss = {} accuracy = {}'.format(epoch, iteration+1, loss, accuracy))
                np.save('data/layers_{}_{}.npy'.format(epoch, iteration+1), layers)

        _, accuracy_test = predict(X_test.T, Y_test.T, layers)
        pred_train, _ = forward_propogation(X_train[:10000].T, layers)
        loss_train = compute_cost(pred_train, Y_train[:10000].T)
        accuracy_train = compute_accuracy(pred_train, Y_train[:10000].T)
        print('Epoch [{}] average_loss = {} average_accuracy = {}'.format(epoch, np.mean(losses), np.mean(accuracies)))
        logger.info('Epoch [{}] average_loss = {} average_accuracy = {}'.format(epoch, np.mean(losses), np.mean(accuracies)))
        print('Epoch [{}] train_loss = {} train_accuracy = {}'.format(epoch, loss_train, accuracy_train))
        logger.info('Epoch [{}] train_loss = {} train_accuracy = {}'.format(epoch, loss_train, accuracy_train))
        print('Epoch [{}] test_accuracy = {}'.format(epoch, accuracy_test))
        logger.info('Epoch [{}] test_accuracy = {}'.format(epoch, accuracy_test))
    
    return layers

In [ ]:
# Create log file
logger = create_logger('output', 'train_log')

# load dataset
X_train, Y_train = load_mnist('data', 'train')
X_test, Y_test = load_mnist('data', 'test')
# normalization for data
X_train = (X_train / 255.0 - 0.5) * 2 # [-1, 1]
X_test = (X_test / 255.0 - 0.5) * 2 # [-1, 1]
# transform the label into one-hot form
(num_train,) = Y_train.shape
Y = np.zeros((num_train, 10))
for i in range(num_train):
    Y[i, Y_train[i]] = 1
Y_train = Y
(num_test,) = Y_test.shape
Y = np.zeros((num_test, 10))
for i in range(num_test):
    Y[i, Y_test[i]] = 1
Y_test = Y

# Construct model of THREE layers
layer1={}
layer1['mode'] = 'fc'
layer1['n_now'] = 300
layer1['n_prev'] = 784
layer2={}
layer2['mode'] = 'fc'
layer2['n_now'] = 150
layer2['n_prev'] = 300
layer3={}
layer3['mode'] = 'fc'
layer3['n_now'] = 10
layer3['n_prev'] = 150
construct_layers = [layer1, layer2, layer3]

In [ ]:
num_experiments = 2
for index in range(num_experiments):
    print('------------------------------------- Experiment {} -------------------------------------'.format(index+1))
    logger.info('------------------------------------- Experiment {} -------------------------------------'.format(index+1))

    initial_layers_path = 'data/initial_layers_{}.npy'.format(index+1)
    if os.path.exists(initial_layers_path):
        initial_layers = np.load(initial_layers_path)
        print('Load initial parameters from {}'.format(initial_layers_path))
        logger.info('Load initial parameters from {}'.format(initial_layers_path))
    else:
        initial_layers = initialize_parameters(construct_layers)
        np.save(initial_layers_path, initial_layers)
        print('Initialize layers and save as {}'.format(initial_layers_path))
        logger.info('Initialize layers and save as {}'.format(initial_layers_path))
    
    layers = train_mini_batch(X_train, Y_train, X_test, Y_test, initial_layers, logger, batch_size=10, num_epoch=3, learning_rate=0.01)